<a href="https://colab.research.google.com/github/arangodb/interactive_tutorials/blob/master/notebooks/Graph_Retail_DA_I.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

%%capture
!git clone -b retail_data_branch --single-branch https://github.com/arangodb/interactive_tutorials.git
!rsync -av  interactive_tutorials/notebooks/data  rsync -av interactive_tutorials/notebooks/tools interactive_tutorials/notebooks/img ./ --exclude=.git
!pip install python-arango
!pip install arangopipe==0.0.70.0.0
!pip install pandas PyYAML==5.1.1 sklearn2
!pip install jsonpickle
!pip3 install networkx
!pip3 install matplotlib
!pip3 install adbnx-adapter
!chmod a+x tools/* 


## Methodology Overview

This notebook provides the descriptive features of the frequent customer group. These features are obtained by applying graph analytic concepts to the frequent customer group. To do so, we need to represent the frequent customer group as a graph. In particular, this group is modeled as a bi-partite graph with the customer nodes and item nodes forming the two disjoint vertex sets. The purchase of an item by a customer as part of an invoice is modeled as an edge.


![](https://github.com/rajivsam/interactive_tutorials/blob/master/notebooks/img/bi_partite_graph.png?raw=1)


This notebook will use ideas from:

1.    [Statistical Analysis of Network Data](https://bigdata.unl.edu/documents/ASA_Workshop_Materials/Tutorial%20Statistical%20Analysis%20of%20Network%20Data.pdf): Provides a methodology for characterizing network structure
2.   [Basic Notions for the Analysis of Affiliate Networks](https://github.com/arangoml/networkx-adapter/blob/master/bipartite_clustering.pdf) : Provides a methodology for analyzing bi-partite graphs

The descriptive characteristics of the graph are captured by:


1.   Vertex and Edge Characteristics
2.   Network Cohesion

These perspectives are important because:


1.   They provide insights about customer behavior or the concept modeled by the graph
2.   They provide the basis for developing other machine learning tasks for the problem. 

For example, we may be interested in the following:
*   How can we develop a generative model for the graph? 
*   How do we model the formation of links in the network?

These tasks tell us how customers make purchases. For details of developing a generative model on bipartite graphs, see [Bipartite Graphs as Models of Complex Networks](http://denif.ens-lyon.fr/data/algorithmique_reseaux_telecoms_enslyon/2007/cours/biparti.pdf) and [Measuring and Modeling Bipartite Graphs with Community Structure](https://www.osti.gov/servlets/purl/1561802). Developing these models requires the descriptive characterization of the graph, which is what is done in this notebook. This chararacterization provides information about properties such as degree distribution and clustering characteristics of nodes which are used to develop the models discussed.

While summary statistics and distributions can be used to provide one descriptive perspective of the network (vertex and edge characteristics), information about grouping characteristics provides the complimentary perspective (network-cohesion). There are multiple ways to describe network-cohension. In this notebook, we will describe two ways to capture or describe cohesive tendencies in the notebook. These are the number of connected components in the graph and measurements describing clique formation in the graph.

The rest of the notebook will provide the implementation of the methodology discussed here. There is one other conceptual idea that we require in the toolbox for the analysis of bi-partite graphs. This is the notion of a _projected graph_.  The two disjoint vertex sets are called the top and bottom vertex sets. The choice of which set to call the top set is arbitrary. It is possible to determine a _projection_ of a set of vertices belonging to either the top or bottom vertex set onto the opposite vertex set. In other words, it is possible to _project_ a set of top vertices on to the bottom vertex set, or, vice versa. The projection of the top set of vertices on to the bottom set, we connect two vertices of the bottom set in the projected graph if they are linked by a vertex in the opposite set. An example graph and the projection graph of the top set of vertices to the bottom set is shown below.


![](https://github.com/rajivsam/interactive_tutorials/blob/master/notebooks/img/projection_graph.png?raw=1)

The customer and the item projection graph have meaningful interpretations. This is discussed when these graphs are generated. The process followed in this notebook to implement this methodology is discussed above is shown below.




![](https://github.com/rajivsam/interactive_tutorials/blob/master/notebooks/img/notebook_3_flow_chart.png?raw=1)

## Read Graph Data

An arango DB graph image from the previous excercise in this series is the starting point for this excercise. We connect to an Oasis instance that will be used for this excercise. 

In [ ]:
from arangopipe.arangopipe_storage.arangopipe_api import ArangoPipe
from arangopipe.arangopipe_storage.arangopipe_admin_api import ArangoPipeAdmin
from arangopipe.arangopipe_storage.arangopipe_config import ArangoPipeConfig
from arangopipe.arangopipe_storage.managed_service_conn_parameters import ManagedServiceConnParam
mdb_config = ArangoPipeConfig()
msc = ManagedServiceConnParam()
conn_params = { msc.DB_SERVICE_HOST : "arangoml.arangodb.cloud", \
                        msc.DB_SERVICE_END_POINT : "createDB",\
                        msc.DB_SERVICE_NAME : "createDB",\
                        msc.DB_SERVICE_PORT : 8529,\
                        msc.DB_CONN_PROTOCOL : 'https',\
                        msc.DB_REPLICATION_FACTOR: 3}
        
mdb_config = mdb_config.create_connection_config(conn_params)
admin = ArangoPipeAdmin(reuse_connection = False, config = mdb_config)
ap_config = admin.get_config()
ap = ArangoPipe(config = ap_config)
proj_info = {"name": "Retail_Graph_Analytics"}
proj_reg = admin.register_project(proj_info)
mdb_config.get_cfg()

In [ ]:
conn = mdb_config.get_cfg()
login = conn['arangodb']

In [ ]:
login

In [ ]:
!pwd

In [ ]:
# Restore an ArangoDB dump into the database.
!./tools/arangorestore  -c none --server.endpoint  http+ssl://{login["DB_service_host"]}:{login["DB_service_port"]} --server.username {login["username"]} \
--server.database {login["dbName"]} \
--server.password {login["password"]} \
--replication-factor 3  \
--input-directory "data/retail_freq_cust_data_dump_eonb2"

## Create Networkx Graph from ArangoDB Graph

For this excercise, we will use the features of the Networkx library to perform the descriptive characterization of the graph. To do so, we will need a Networkx representation of the frequent customer graph. This can be obtained using the Networkx Adapter. To create the networkx graph, we need to provide the Networkx adapter with the following information:
1. Connection information to the ArangoDB database that contains the graph that we want to covert to a Networkx graph
2. Structural information to create the graph. A description of the vertex and edge collections of the graph in ArangoDB need to be provided.

When this information is supplied to the adapter, the adapter creates an instance of a NetworkX graph and returns it. This instance is used for the rest of the analysis in this excercise.


In [ ]:
import networkx as nx
from adbnx_adapter.arangoDB_networkx_adapter import ArangoDB_Networkx_Adapter

# Specify the connection to the ArangoDB Database
con = {'dbName': login["dbName"],
 'username': login["username"],
 'password': login["password"],
 'hostname': login["DB_service_host"],
 'port': login["DB_service_port"]}

# Create Adapter instance
ma = ArangoDB_Networkx_Adapter(conn = con)

freq_customers = {'vertexCollections': {'Customers': {},
                                         'Items': {}},
                   'edgeCollections': {'Purchases': {'_from', '_to', 'spend'}}}

# Export networkX graph                                  
g = ma.create_networkx_graph(graph_name = '',  graph_attributes = freq_customers)


## Generate Vertex and Edge Characteristics

In this section, the vertex and edge characteristics of frequent customer bi-partite graph are computed.

To begin with we will compute the summary statistics of the bi-partite graph and we will generate the degree distribution of the two vertex sets, the customer and the item vertex sets for this graph. The details of performing this with Networkx are shown below. 




### Compute summary statistics for the bi-partite graph

In [ ]:
gp = g.to_undirected()

In [ ]:
cust_nodes = [n for n in g.nodes() if n.startswith("Customers")]
item_nodes = [n for n in g.nodes() if n.startswith("Items")]

In [ ]:
len(cust_nodes)

In [ ]:
len(item_nodes)

In [ ]:
len(g.edges())

In [ ]:
gp.is_directed()

### Compute Distributions of Customer and Item Vertex Sets
We first create a bi-partite version of the graph in networkx. The earlier graph did not discriminate the type of the node. We do that now using the bi-partite graph abstraction available in networkx.

In [ ]:
import networkx as nx
B = nx.Graph()
B.add_nodes_from(cust_nodes, bipartite=0)
B.add_nodes_from(item_nodes, bipartite=1)
B.add_edges_from(list(gp.edges()))

In [ ]:
from networkx.algorithms import bipartite
degc, degi = bipartite.degrees(B, item_nodes)
dc = [v for k,v in degc]
di = [v for k,v in degi]

#### Customer Degree Distribution

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
sns.set(rc={'figure.figsize':(11.7,8.27)})
ax = sns.distplot(dc, hist = True)
plt.title("Distribution of Customer Degree ")
plt.xlabel("Customer Degree")

#### Item Degree Distribution

In [ ]:
sns.set(rc={'figure.figsize':(11.7,8.27)})
ax = sns.distplot(di, hist = True)
plt.title("Distribution of Item Degree ")
plt.xlabel("Item Degree")

In [ ]:
ci = bipartite.clustering(B, item_nodes)

In [ ]:
cv = [v for k, v in ci.items()]

In [ ]:
sns.set(rc={'figure.figsize':(11.7,8.27)})
ax = sns.distplot(cv, hist = True)
plt.title("Distribution of Item Clustering ")
plt.xlabel("Item Clustering Coefficient")

### Observations


1.   Random networks (Erdos-Renyi) do not display clustering tendencies. An examination of the clustering coefficients associated with the item nodes clearly shows clustering tendencies. Therefore, this elminates a random graph as a possible genrative model for this graph.
2.   The degree distribution for the customer and item nodes are non-monotonic as would be observed if either of these nodes show scale-free or power law behavior. In such a case a small set of nodes will act as hubs and have very high degress while most nodes have low degrees. Instead what we observe is that most nodes have similar degrees and a small set of nodes corresponding to both tails have either small or high degrees. This is consistent with _small world networks_ .
3. This implies that most customers buy about the same range of items. For example, a review of the item degree graph shows that most items are bought between 25-75 people, but there a few items that are bought by many people and a few items that are not bought by many people.



## Projection Graphs
The other perspective in characterizing bipartite graphs are the characteristics of the projection graphs. The notion of the projection graph was described in the methodological overview section. We will now generate this projections for the frequent shopper group.

In [ ]:
cust_graph = nx.bipartite.projected_graph(B, cust_nodes)
item_graph = nx.bipartite.projected_graph(B, item_nodes)

### Observations
In the customer projection graph, two nodes are connected if they have purchased an item in common. Similarly, in the item projection graph, two items are connected if they have been purchased by a customer. This implies


1.   The customer projection graph captures shared customer taste in purchases. If we pick any random node in the customer projection graph, then the users who are connected by an edge to him or her are those who share at least one purchase. A user who does not share an edge with a user is one whose shopping preferences are different. 
2.   The item projection graph captures copurchased behavior in items. If two items are connected in the item projection graph, it implies that these items were purchased by a customer. If we pick a random item in the item projection graph, following the edges connected to it tells us the items purchased with it. Items not connected to it are not purchased with it.




#### Degree distribution of Customer Graph

In [ ]:
dc = nx.degree(cust_graph)
dcv = [ v for k, v in dc]

### Max Customer Projection Graph Degree

In [ ]:
max(dcv)

### Observation
There is a customer whose purchasing habbits match 421 other customers!

### Min Customer Projection Graph Degree

In [ ]:
min(dcv)

### Observation
There are customers whose purchasing habbits don't match any of the others in the group. These would be the outliers discussed later.

In [ ]:
import numpy as np
np.sum(dcv)/cust_graph.number_of_nodes()

Observation
On average a user shares taste with 287 other users. A lot of shared taste. We should expect to see cliques in the graph

In [ ]:
sns.set(rc={'figure.figsize':(11.7,8.27)})
ax = sns.distplot(dcv, hist = True)
plt.title("Degree distribution of Customer Graph ")
plt.xlabel("Degree Distribution of Customer Projection Graph")

#### Observation
The frequent shopper group has a very high degree of shared taste. If we pick a random user in the graph, he or she is connected to hundreds (the peak density is at around 350) of other users. This implies they have a similar taste in products they purchase.

#### Degree Distribution of the Item Projection Graph

In [ ]:
di = nx.degree(item_graph)
div = [ v for k, v in di]

### Max Item Projection Graph Degree

In [ ]:
max(div)

### Observation
There is an item that is bought with 74 of the 75 items

### Min Item Projection Graph Degree

In [ ]:
min(div)

### Observation
There is an item that is purchased with 66 of the 75 items purchased by the frequent shopper group. This suggests that there is a lot of shared taste in the frequent shopper group since at a minimum an item is purchased with 66 of the 75 inventory items.

In [ ]:
sns.set(rc={'figure.figsize':(11.7,8.27)})
ax = sns.distplot(div, hist = True)
plt.title("Degree distribution of Item Projection Graph ")
plt.xlabel("Degree Distribution of Item Projection Graph")

#### Observation
Since the customer and item projection graph are highly interconnected and since we know we are not working with a power law graph, we don't explore degree based centrality measures. Betweenness and Eigenvector centrality have useful interpretations


In [ ]:
evc_cust = nx.eigenvector_centrality(cust_graph)

In [ ]:
import operator
sorted_evc = dict( sorted(evc_cust.items(), key=operator.itemgetter(1),reverse=True))

In [ ]:
i = 0
for k, v in sorted_evc.items():
  evcs = "Customer %s , eigenvalue centrality %.3f" %(k,v)
  print(evcs)
  i = i + 1
  if i == 10:
    break 


Observation: These customers are important because they are connected to customers who make a lot of purchases.

In [ ]:
bc_cust = nx.betweenness_centrality(cust_graph)

In [ ]:
import operator
sorted_bc = dict( sorted(bc_cust.items(), key=operator.itemgetter(1),reverse=True))

In [ ]:
i = 0
for k, v in sorted_bc.items():
  bcs = "Customer %s , betweeness centrality %.3f" %(k,v)
  print(bcs)
  i = i + 1
  if i == 10:
    break 


Observation: These customers are important because they figure in the paths connecting customers. If these customers make new purchases, then their network could follow.

In [ ]:
evc_item = nx.eigenvector_centrality(item_graph)

In [ ]:
import operator
sorted_evi = dict( sorted(evc_item.items(), key=operator.itemgetter(1),reverse=True))

In [ ]:
i = 0
for k, v in sorted_evi.items():
  evis = "Item %s , eigenvalue centrality %.3f" %(k,v)
  print(evis)
  i = i + 1
  if i == 10:
    break 


Observation: These items are important because they are connected to items purchased a lot.

In [ ]:
bc_item = nx.betweenness_centrality(item_graph)

In [ ]:
import operator
sorted_bc = dict( sorted(bc_item.items(), key=operator.itemgetter(1),reverse=True))

In [ ]:
i = 0
for k, v in sorted_bc.items():
  bcs = "Item %s , betweeness centrality %.6f" %(k,v)
  print(bcs)
  i = i + 1
  if i == 10:
    break 

Observation: As will be discussed in the section on Network cohesion, there are large cliques in the item projection graph. The notion of betweeness does not offer a useful interpretation with this structure because most items are co-purchased with other items

## Network Cohesion

We described the vertex and edge characteristics of the graph representation associated with the frequent shopper group. In doing so, we already discovered that this group is homogeneous with a lot of shared taste. The other perspective to characterize the graph is a description of cohesion. We will do that in this section.

#### Connected Components
A fundamental question to determine the answer to is: "Is the customer projection graph connected?" This asks if we starting with any user we can reach another user in the projection graph. If we can reach this user in one hop, it means the user shares shopping preference with the user we are starting with. If we need two hops, it means there is no shared taste, but someone who the starting user shares taste with shares taste with this user. Can every customer in the frequent shopper group be related this way? Are there exceptions? We can find the answer by determining the connected components of the customer projection graph. This done below.

##### Check if the customer projection graph is connected

In [ ]:
nx.is_connected(cust_graph)

#### Idenfify the connected components for the customer projected graph

In [ ]:
[len(c) for c in sorted(nx.connected_components(cust_graph), key=len, reverse=True)]

### Store the Customer and Item Projected Graph for Analysis

These graphs will be analyzed in the next excercise.

In [ ]:
df_cgraph_comp1 = nx.to_pandas_edgelist(cust_graph.subgraph(next(nx.connected_components(cust_graph))))
fp = "data/customer_projected_graph.csv"
df_cgraph_comp1.to_csv(fp, index = False)
df_cgraph_comp1.head()

In [ ]:
df_item_graph = nx.to_pandas_edgelist(item_graph)
fp = "data/item_projected_graph.csv"
df_item_graph.to_csv(fp, index = False)
df_item_graph.head()

### Observation
There is a large group of customers (437) whose shopping tastes are related and a group of ten customers who are outliers. These ten customers have shopping taste that is different from the other group with related taste.


#### Check if the item projection graph is connected

In [ ]:
nx.is_connected(item_graph)

##### Observation:
This implies that item group can be related and there are no outliers like with the customers.

### K Core 
Since the item and customer graph have high degrees associated with most nodes, we are likely to find cliques in both these projection graph. A clique is a graph where each node is conneted to the other nodes in the graph. A clique in the customer projection graph represent customers who buy the same items. The size of the clique captures the items they buy in common. Analogously, a clique in the item projection graph represents items that are purchased together. A clique can be identified using a K-Core of the graph. A K-Core of the graph represents a subgraph where the degree of each degree is at least K. To identify the largest clique we will start with the K set to the maximum degree and check if we can find a K core, if we cannot find a clique, we decrement K till we reach a K where a K core is found. The segments below illustrate this process for both the item and customer projection graphs.


In [ ]:
k = 400
while True:
  kg = nx.k_core(cust_graph, k)
  if kg.number_of_nodes() > 0:
    break
  else:
    k = k -1


In [ ]:
print("Largest clique in customer graph is {} ".format(k))

### Observation:
We can find a group of users where each user is similar to 224 users in terms of purchasing habbits.

In [ ]:
kg.number_of_nodes()

In [ ]:
i = 75
while True:
  kg = nx.k_core(item_graph, i)
  if kg.number_of_nodes() > 0:
    break
  else:
    i = i -1

In [ ]:
print("Largest clique in item graph is {} ".format(i))

### Observation:
We can find a group of 67 items in the store that are purchased together.

## Summary of Identified Facts



*   The customer and item degree distributions show non-monotonic behavior. Most nodes have similar degrees. A small group of nodes have high and low degrees. This is suggestive of small world behavior. If the degree distributions showed a monotonic character with a small number of nodes having high degrees (hubs) with others having smaller degrees, that would suggest a power law network. Since we observe clustering tendencies in the graph we know that the graphs are not random graphs (which have no clustering tendencies).
*   The customer and item projection graphs represent shared shopping taste and co-purchasing behavior. Shared taste implies users buy similar items. Copurchasing behavior captures items that are purchased together. Both of these behaviors are critical information to the business.
*  Users adjacent to a customer in the customer projection graph are the users with similar to the user.
* Users non-adjacent (more than one hop away) are disimilar to the user.
* Items adjacent to an item in the item projection graph are the items co-purchased with the items.
* Items not adjacent (more than one hop away) are not purchased with the item.
* Diameter of the user projection graph identifies the most disimilar users in terms of taste.
* Diameter of the item graph identifies items that are most unlikely to be purchased together.
* There are a group of 10 customers with unique tastes. They are the outliers in the customer group.
* Most nodes in the customer and item projection graph have high degrees. This implies that these graphs have cliques.
* Cliques in the customer graph identify cohorts of customers with shared taste. The clique size can be used to specify the shared taste.
* Cliques in the item projection identify cohorts of items purchased together. The clique size can be used to quantify the co-purchasing behavior


## Log the Modeling Activity in Arangopipe

In [ ]:
ds_info = {"name" : "Frequent_Customer_Graph",\
            "description": "Frequent Customer Graph of Retail Dataset",\
           "source": "ArangoDB dump of Retail from Notebook II" }
ds_reg = ap.register_dataset(ds_info)

In [ ]:

featureset = {"name": "Features are attributes of the nodes"}
fs_reg = ap.register_featureset(featureset, ds_reg["_key"]) 

In [ ]:
import io
import requests
url = ('https://raw.githubusercontent.com/arangodb/interactive_tutorials/master/notebooks/Graph_Retail_Descriptive_Analytics.ipynb')
nbjson = requests.get(url).text

In [ ]:

model_info = {"name": "Descriptive Analytics for Frequent Customers",  "task": "Exploratory Data Analysis", 'notebook': nbjson}
model_reg = ap.register_model(model_info, project = "Retail_Graph_Analytics")

In [ ]:
import uuid
import datetime
#import jsonpickle

ruuid = str(uuid.uuid4().int)
model_perf = {'run_id': ruuid, "timestamp": str(datetime.datetime.now())}

#mp = jsonpickle.encode(mp)
model_params = {'run_id': ruuid, 'model_params': 'Not Applicable'}

run_info = {"dataset" : ds_reg["_key"],\
                    "featureset": fs_reg["_key"],\
                    "run_id": ruuid,\
                    "model": model_reg["_key"],\
                    "model-params": model_params,\
                    "model-perf": model_perf,\
                    "tag": "Retail Descriptive Analytics Notebook",\
                    "project": "Retail_Graph_Analytics"}
ap.log_run(run_info)

In [ ]:
!./tools/arangodump  -c none --server.endpoint http+ssl://{login["DB_service_host"]}:{login["DB_service_port"]} --server.username {login["username"]} --server.database {login["dbName"]} --server.password {login["password"]} --overwrite true --output-directory "data/retail_freq_cust_data_dump_eonb3"